In [ ]:
import numpy as np
from numpy import genfromtxt
from numpy import savetxt

In [ ]:
# import the necessary packages
import numpy as np
class NeuralNetwork:

 def __init__(self, layers, alpha=0.1):
    # initialize the list of weights matrices, then store the
    # network architecture and learning rate
    self.W = []
    self.layers = layers
    self.alpha = alpha
    # start looping from the index of the first layer but
    # stop before we reach the last two layers
    for i in np.arange(0, len(layers) - 2):
    
      # randomly initialize a weight matrix connecting the
      # number of nodes in each respective layer together,
      # adding an extra node for the bias
      w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
      self.W.append(w / np.sqrt(layers[i]))
  

    # the last two layers are a special case where the input
    # connections need a bias term but the output does not
    w = np.random.randn(layers[-2] + 1, layers[-1])
    self.W.append(w / np.sqrt(layers[-2]))
    
 def __repr__(self):
		# construct and return a string that represents the network
		# architecture
		return "NeuralNetwork: {}".format(
			"-".join(str(l) for l in self.layers))
  
 def sigmoid(self, x):
		return 1.0 / (1 + np.exp(-x))
  
 def sigmoid_deriv(self, x):
    return x * (1 - x)

 def propagate(self, X, y, epochs=1000, Update=100):
		# insert a column of 1's as the last entry in the feature
		# matrix -- this little trick allows us to treat the bias
		# as a trainable parameter within the weight matrix
		X = np.c_[X, np.ones((X.shape[0]))]


		for epoch in np.arange(0, epochs):
			# loop over each individual data point and train
			# our network on it
			for (x, target) in zip(X, y):
				self.backpropagation(x, target)
    
			# check to see if we should display a training update
			if epoch == 0 or (epoch + 1) % Update == 0:
				loss = self.calculate_loss(X, y)
				print("[INFO] epoch={}, loss={:.7f}".format(epoch + 1, loss))
  
 def backpropagation(self, x, y):
		# constructed a list of output activations for each layer
		# as our data point flows through the network; the first
		# activation is a special case;it's just the input
		# feature vector itself
		A = [np.atleast_2d(x)]

		# loop over the layers in the network
		for layer in np.arange(0, len(self.W)):
			# feedforward the activation at the current layer by
			# taking the dot product between the activation and
			# the weight matrix; this is called the "net_linear input"
			# to the current layer
			net_linear = A[layer].dot(self.W[layer])
			# computing the "net Non_linear output" is simply applying our
			# nonlinear activation function to the net_linear input
			non_linear_out = self.sigmoid(net_linear)
			# once we have the net output, add it to our list of
			# activations
			A.append(non_linear_out)
   
    		
		# the first phase of backpropagation is to compute the
		# difference between our *prediction*  and the true target
		# value
		error = A[-1] - y

		# apply the chain rule and build our
		# list of deltas 'D'
		Error_array = [error * self.sigmoid_deriv(A[-1])]
 
		for layer in np.arange(len(A) - 2, 0, -1):
			delta = Error_array[-1].dot(self.W[layer].T)
			delta = delta * self.sigmoid_deriv(A[layer])
			Error_array.append(delta)
    # since we looped over our layers in reverse order we need to
		# reverse the deltas
		Error_array = Error_array[::-1]

		# loop over the layers
		for layer in np.arange(0, len(self.W)):
			# update our weights by taking the dot product of the layer
			# activations with their respective deltas, then multiplying
			# this value by some small learning rate and adding to our
			# weight matrix 
			self.W[layer] += -self.alpha * A[layer].T.dot(Error_array[layer])

 def predict(self, X, addBias=True):
		p = np.atleast_2d(X)
  
		# check to see if the bias column should be added
		if addBias:
			# insert a column of 1's as the last entry in the feature
			# matrix (bias)
			p = np.c_[p, np.ones((p.shape[0]))]

		for layer in np.arange(0, len(self.W)):

			p = self.sigmoid(np.dot(p, self.W[layer]))
		return p

 def calculate_loss(self, X, targets):
		# make predictions for the input data points then compute
		# the loss
		targets = np.atleast_2d(targets)
		predictions = self.predict(X, addBias=False)
		loss = 0.5 * np.sum((predictions - targets) ** 2)
		# return the loss
		return loss

 def save(self):
    np.save('weights.npy', self.W)
    pass
  
  
    

    

In [ ]:
# Converting the files to numpy array
train_dataset = genfromtxt('/content/drive/MyDrive/ECE657/train_data.csv', delimiter=',') # Replace this with your data

In [ ]:
# Converting the files to numpy array
train_label = genfromtxt('/content/drive/MyDrive/ECE657/train_labels.csv', delimiter=',') # Replace this with your data

In [ ]:
# Obtained the sum of each rows
sum_of_rows = train_dataset.sum(axis=1)

In [ ]:
# Normalized each row using their sums
norm_train_dataset = train_dataset / sum_of_rows[:, np.newaxis]

In [ ]:
# Here our goal is to split the data into train data and validation data 

def split_data(X, y):
    #Created a dummy array with the size of the dataset 
    arr_rand = np.random.rand(X.shape[0])

    #This creates a randomly populated array of boolean variables the length 
    # of the arr_rand
    split = arr_rand < np.percentile(arr_rand, 70)
    #Here the spliting for test and train is carried out using the 
    #array of boolean variables 
    X_train = X[split]  
    y_train = y[split]
    X_test =  X[~split]
    y_test =  y[~split]

    # print(f"{len(X_train)}, {len(y_train)}, {len(X_test)}, {len(y_test)}")
    return X_train, y_train, X_test, y_test

In [ ]:
x_train, y_train, X_test , y_test = split_data(norm_train_dataset, train_label)

In [ ]:
# savetxt('test_X.csv', X_test, delimiter=',')
# savetxt('test_y.csv', y_test, delimiter=',')

In [ ]:
# Initialization of the neural network
nn = NeuralNetwork([x_train.shape[1], 50 , 4])
print("[BIO] {}".format(nn))

In [ ]:
nn.save()

In [ ]:
nn.propagate(x_train, y_train, epochs=1000)

In [ ]:
# # evaluate the network
# print("[LOG] Network Evaluation")
# predictions = nn.predict(X_test)
# predictions = predictions.argmax(axis=1)
# print(classification_report(y_test.argmax(axis=1), predictions))

In [ ]:
# len(X_test)

In [ ]:
# now that our network is trained, loop over the XOR data points
prediction = []
actual = []
for (x , target) in zip(X_test , y_test):
  # make a prediction on the data point 
  pred = nn.predict(x)
  prediction.append(np.argmax(pred,axis=1))
  actual.append(np.argmax(target,axis = 0))


In [ ]:
correct = 0
for i in range(0 , len(actual)):
  # comparing individual prediction with the target 
  
  if actual[i] == prediction[i]:
    correct+=1
# we then used the value obtained to calculate the accuracy
print(f"{int((correct/len(actual)) * 100)}%")